<h6>Slicing</h6>
<p>The convention is to use start:end:increment for indexing using colon operators in python
        array/lists. If you have a 1d array then slice a sgement using the notation above. If there is 
        nothing before the colon it means start from the beginning. A [:1] means start from the beginning and
        end at index 1 so this returns the first element. A [1:] means start at the first element and return the
    rest of the array. 
</p>



In [137]:


#index shifting
d = [1,2,3,4,5,6]
display(d)
#shift right
display("original d array")
display("from index 1 to end")
display(d[1:len(d)])
#shorthand
display(d[1:])
display("from start to end-1, len(arr)-1 shortened to -1")
#shift left
display(d[:len(d)-1])
#the -1 index is short for len(arr)-1 
display(d[:-1])
#for a 2d array we have the same conventions
td = [[1,2],[3,4],[5,6]]
td[:-1]

[1, 2, 3, 4, 5, 6]

'original d array'

'from index 1 to end'

[2, 3, 4, 5, 6]

[2, 3, 4, 5, 6]

'from start to end-1, len(arr)-1 shortened to -1'

[1, 2, 3, 4, 5]

[1, 2, 3, 4, 5]

In [131]:
import tensorflow as tf
import numpy as np
import pandas
from IPython.display import display

d = [1,2,3,4,5,6,1,2,3,4,5,6]
df = pandas.DataFrame(d)
display(df.head())
df_shift = df.shift(1)

display(df_shift.head())

#what to convert NaN to? What is the shifted one at the end supposed to look like? 
d_2=[[1,2],[3,4],[5,6],[7,8]]
df_2 = pandas.DataFrame(d_2)
display(df_2.head())
df_2shift = df_2.shift(1,axis=1)
display(df_2shift.head())



,0
0,1
1,2
2,3
3,4
4,5


,0
0,NaN
1,1.0
2,2.0
3,3.0
4,4.0


,0,1
0,1,2
1,3,4
2,5,6
3,7,8


,0,1
0,NaN,1.0
1,NaN,3.0
2,NaN,5.0
3,NaN,7.0


'original d array'

...

[1, 2, 3, 4, 5]

'd_leftlong'

[1, 2, 3, 4, 5]

'indexing from end'

[6, 5, 4, 3, 2, 1]

In [138]:
import numpy as np
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
import time
import os
import urllib.request
from tensorflow.models.rnn.ptb import reader

In [142]:
file_url = 'https://raw.githubusercontent.com/jcjohnson/torch-rnn/master/data/tiny-shakespeare.txt'
file_name = 'tinyshakespeare.txt'
if not os.path.exists(file_name):
    urllib.request.urlretrieve(file_url, file_name)

with open(file_name,'r') as f:
    raw_data = f.read()
    print("Data length:", len(raw_data))
vocab = set(raw_data)
vocab_size = len(vocab)
idx_to_vocab = dict(enumerate(vocab))
vocab_to_idx = dict(zip(idx_to_vocab.values(), idx_to_vocab.keys()))

data = [vocab_to_idx[c] for c in raw_data]
del raw_data

def gen_epochs(n, num_steps, batch_size):
    for i in range(n):
        yield reader.ptb_iterator(data, batch_size, num_steps)

def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

def train_network(g, num_epochs, num_steps = 200, batch_size = 32, verbose = True, save=False):
    tf.set_random_seed(2345)
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        training_losses = []
        for idx, epoch in enumerate(gen_epochs(num_epochs, num_steps, batch_size)):
            training_loss = 0
            steps = 0
            training_state = None
            for X, Y in epoch:
                steps += 1

                feed_dict={g['x']: X, g['y']: Y}
                if training_state is not None:
                    feed_dict[g['init_state']] = training_state
                training_loss_, training_state, _ = sess.run([g['total_loss'],
                                                      g['final_state'],
                                                      g['train_step']],
                                                             feed_dict)
                training_loss += training_loss_
            if verbose:
                print("Average training loss for Epoch", idx, ":", training_loss/steps)
            training_losses.append(training_loss/steps)

        if isinstance(save, str):
            g['saver'].save(sess, save)

    return training_losses




def build_basic_rnn_graph_with_list(
    state_size = 100,
    num_classes = vocab_size,
    batch_size = 32,
    num_steps = 200,
    learning_rate = 1e-4):

    reset_graph()

    x = tf.placeholder(tf.int32, [batch_size, num_steps], name='input_placeholder')
    y = tf.placeholder(tf.int32, [batch_size, num_steps], name='labels_placeholder')

    x_one_hot = tf.one_hot(x, num_classes)
    rnn_inputs = [tf.squeeze(i,squeeze_dims=[1]) for i in tf.split(1, num_steps, x_one_hot)]

    cell = tf.nn.rnn_cell.BasicRNNCell(state_size)
    init_state = cell.zero_state(batch_size, tf.float32)
    rnn_outputs, final_state = tf.nn.rnn(cell, rnn_inputs, initial_state=init_state)

    with tf.variable_scope('softmax'):
        W = tf.get_variable('W', [state_size, num_classes])
        b = tf.get_variable('b', [num_classes], initializer=tf.constant_initializer(0.0))
    logits = [tf.matmul(rnn_output, W) + b for rnn_output in rnn_outputs]

    y_as_list = [tf.squeeze(i, squeeze_dims=[1]) for i in tf.split(1, num_steps, y)]

    loss_weights = [tf.ones([batch_size]) for i in range(num_steps)]
    losses = tf.nn.seq2seq.sequence_loss_by_example(logits, y_as_list, loss_weights)
    total_loss = tf.reduce_mean(losses)
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)

    return dict(
        x = x,
        y = y,
        init_state = init_state,
        final_state = final_state,
        total_loss = total_loss,
        train_step = train_step
    )


Data length: 1115394


In [143]:
g = build_basic_rnn_graph_with_list()
t = time.time()
train_network(g, 3)
print("It took", time.time() - t, "seconds to train for 3 epochs.")

TypeError: Input 'split_dim' of 'Split' Op has type float32 that does not match expected type of int32.

In [92]:
import tensorflow as tf
import numpy as np

with open('anna.txt') as file_handle:
    text = file_handle.read()
vocab = set(text)
vocab_to_int = {v:k for k,v in enumerate(vocab)}
int_to_vocab = {k:v for k,v in enumerate(vocab)}
int_to_vocab2 = dict(enumerate(vocab))

#well the first one is easier to remember but the ctor is cool also.
#for k in vocab_to_int:
#    print(k,vocab_to_int[k])
    
l1 = list(int_to_vocab.keys())
print(l1[0],int_to_vocab[10])

l2 = list(int_to_vocab2.keys())
print (l2[0],int_to_vocab2[10])

print(len(int_to_vocab.values()))
print(len(int_to_vocab.keys()))

#what is an encoding? int to letters. 

encoded = [vocab_to_int[c] for c in text]
for c in text:
    print("c:",c," vocab_to_int:",vocab_to_int[c])
    break

        

print("encoded",encoded[0]) #unclear what this does besides turning into nparray and ints. which they are. 
print("first 100 chars:",text[:100])
encoded[:100]#is a list not an np.array is this important? 
np.array(encoded[:100])

#Y is input shifted over 1. 


0 a
0 a
83
83
c: C  vocab_to_int: 24
encoded 24
first 100 chars: Chapter 1


Happy families are all alike; every unhappy family is unhappy in its own
way.

Everythin


array([24, 60, 10, 74, 33, 67, 41, 71,  5, 40, 40, 40, 80, 10, 74, 74, 47,
       71, 51, 10, 29, 61, 69, 61, 67, 52, 71, 10, 41, 67, 71, 10, 69, 69,
       71, 10, 69, 61, 27, 67,  2, 71, 67,  6, 67, 41, 47, 71, 31, 28, 60,
       10, 74, 74, 47, 71, 51, 10, 29, 61, 69, 47, 71, 61, 52, 71, 31, 28,
       60, 10, 74, 74, 47, 71, 61, 28, 71, 61, 33, 52, 71, 37, 56, 28, 40,
       56, 10, 47, 50, 40, 40, 13,  6, 67, 41, 47, 33, 60, 61, 28])

In [111]:
#making batches from sequences. 

#each batch contains multiple sequences
#each sequence contains steps. How long is a batch? 
#so confusing
#each batch is a series of sequneces of windows. Window size
#is steps? 

# n_seqs * n_steps = chars per batch. 
#
input = np.array([1,2,3,4,5,6,1,2,3,4,5,6])
n_seqs = 3
n_steps = 2

num_in_batch = n_seqs * n_steps

#how to put sliding window into input_arr
num_batches = len(input) / num_in_batch

#pick num rows first, -1 lets system decide and fill in rest
input_batch = input.reshape(3,-1)
print(input_batch)
print("shape:",input_batch.shape[1])

for n in range(0,input_batch.shape[0],n_steps):
    print(n)
    x=input_batch[:,n:n+n_steps]
    print("x:",x)

y = np.zeros_like(input_batch)
print("y:",y)
y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
print("y:",y)
#for n in range(0, arr.shape[1], n_steps):
#        # The features
#        x = arr[:, n:n+n_steps]
#        # The targets, shifted by one
#        y = np.zeros_like(x)
#        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
#        yield x, y




[[1 2 3 4]
 [5 6 1 2]
 [3 4 5 6]]
shape: 4
0
x: [[1 2]
 [5 6]
 [3 4]]
2
x: [[3 4]
 [1 2]
 [5 6]]
y: [[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
y: [[4 4 4 3]
 [2 2 2 1]
 [6 6 6 5]]


In [1]:
import tensorflow as tf
#source
#https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/recurrent_network.py

from tensorflow.contrib import rnn

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [44]:
class RNN(object):
    def __init__(self):
        self.learning_rate=0.001
        self.training_steps=10000
        self.batch_size=120
        self.display_step=200
        self.num_input=28
        self.timesteps=28
        self.num_hidden=128
        self.num_classes=10
    
    def input(self):
        self.X = tf.placeholder(tf.float32, [None, self.timesteps, self.num_input])
        self.Y = tf.placeholder(tf.float32, [None,self.num_classes])
        self.weights = tf.Variable(tf.random_normal([self.num_hidden, self.num_classes]))
        self.bias = tf.Variable(tf.random_normal([self.num_classes]))
    
    def model(self):
        self.x = tf.unstack(self.X, self.timesteps, 1)
        #print ("x unstack:",self.x.shape)
        with tf.variable_scope('lstm1'):
            self.lstm_cell = tf.contrib.rnn.BasicLSTMCell(self.num_hidden,forget_bias=1.0,reuse=True
                                                         )
            self.outputs,self.states = tf.contrib.rnn.static_rnn(self.lstm_cell, self.x, dtype=tf.float32)
            self.linear_model = tf.matmul(self.outputs[-1], self.weights['out'])+self.biases['out']
            self.logits = tf.nn.softmax(self.linear_model)
    
    def train(self):
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.model,labels=self.Y))
        self.optimizer = tf.train.GradientDescentOptimizer(self.learning_rate)
        self.train_op = self.optimizer.minimize(self.loss) 
        self.correct_pred = tf.equal(tf.argmax(self.logits,1),tf.argmax(self.Y,1))
        self.accuracy = tf.reduce_mean(tf.case(self.correct_pred,tf.float32))
        
        init=tf.global_variables_initializer()
        with tf.Session() as sess:
            sess.run(init)
            for step in range(1,self.training_steps):
                batch_x, batch_y = mnist.train.next_batch(batch_size)
                #reshape for sequential format in a CNN we reshaped to single 768 pixel array
                batch_x = batch_x.reshape(batch_size, timesteps, num_input)
                sess.run(self.train_op, feed_dict={self.X:batch_x, self.Y:batch_y})
                loss,acc = sess.run([loss,accuracy], feed_dict={self.X:batch_x, self.Y:batch_y})
                if(step%100==0):
                    print("Step:",step," loss:", loss, " acc:",acc)
            print("optimization finished")
            test_len = 128
            test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
            test_label = mnist.test.labels[:test_len]
            print("Testing Accuracy:", \
            sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

In [45]:
rnn = RNN()
rnn.input()
rnn.model()
rnn.train()


TypeError: list indices must be integers or slices, not tuple